In [ ]:
! pip install huggingface_hub
! git config --global credential.helper store

In [27]:
from huggingface_hub import notebook_login
from huggingface_hub import HfFolder
import pandas as pd


In [ ]:
#enter your API key, you can make one for free on HF
notebook_login()

In [28]:
from huggingface_hub import InferenceApi

inference = InferenceApi("bigscience/bloom",token=HfFolder.get_token())

In [ ]:
exp_condition_df = pd.read_csv('../experiments/analysis_of_prior_work/aopw_tasks_collection_essentialism.csv')
exp_condition_df = exp_condition_df.fillna("")
exp_condition_df


In [57]:
def bloom_infer(prompt, seed=2022):
    params = {
        "max_new_tokens": 5,
        "top_k": None,
        "top_p": 0.9,
        "temperature": 0.7,
        "do_sample": True,
        "seed": seed,
        "early_stopping": False,
        "no_repeat_ngram_size": None,
        "num_beams": None,
        "eos_token_id": None,
        "length_penalty": 1.0,
    }
    
    response = inference(prompt, params=params)

    return response

In [ ]:

bloom_response_df = pd.DataFrame(columns=exp_condition_df.columns)

for exp_condition in range(...,...): #range(2): #range[1] means the second row of the csv file, can change to range[2] to get the third row of the csv file. We will want to start with [1] since we ran the first row already.
    for rep_id in range(50): # generate 50 responses
      exp_row = exp_condition_df.loc[[exp_condition]]
      exp_row["rep_id"] = rep_id + 1

      prompt = ""
      for scenario_id in range(1, 3):  # We have two scenarios.
          scenario_column_name = f"scenario_{scenario_id}"  # f"scenario_{scenario_id}" is scenario column name
          # print(exp_condition_df[scenario_column_name][exp_condition])
          if exp_condition_df[scenario_column_name][exp_condition] == "":
            break  # Skip when the scenario is empty.
          prompt += exp_condition_df[scenario_column_name][exp_condition].strip()

          for question_id in [1]:            #range(1,7): #some cases have more than one question, so we need to loop through them. If there are none, we will break.
            if scenario_id == 1:
              continue
            question_column_name = f"question_{scenario_id}_{question_id}_text"  # f"question_{scenario_id}_{question_id}_text" is question column name
            if question_column_name not in exp_condition_df:  
              break
            if exp_condition_df[question_column_name][exp_condition] == "":
              break

            if scenario_id == 1:
              prompt += " " + exp_condition_df[question_column_name][exp_condition].strip() + " I think it will be " # need to manually change the "I think it will be " sentence according to the questions
            # example: prompt += " " + exp_condition_df[question_column_name][exp_condition].strip() + " I think it was a  "
            if scenario_id == 2:
              prompt += " " + exp_condition_df[question_column_name][exp_condition].strip() + " I think the thing will be " # need to manually change the "I think the thing will be " sentence according to the questions
    
            else:
              prompt += " " + exp_condition_df[question_column_name][exp_condition].strip() + " I think it would be " # need to manually change the " I think it would be " sentence according to the questions
    
            
            exp_row[f"bloom_prompt_question{scenario_id}_{question_id}"] = prompt
            print(prompt)
            response = bloom_infer(prompt, seed=rep_id)
            answer = response[0]["generated_text"][len(prompt):].strip()
            print(answer)

            bloom_response_column_name = f"bloom_response_question_{scenario_id}_{question_id}" 
            exp_row[bloom_response_column_name] = answer

            prompt += answer + "\n"

      bloom_response_df = pd.concat([bloom_response_df, exp_row], axis=0)

bloom_response_df.rep_id = bloom_response_df.rep_id.astype(int)

col = bloom_response_df.pop("rep_id")
bloom_response_df.insert(2, col.name, col)

bloom_response_df



In [59]:
bloom_response_df.to_csv("../../data/analysis_of_prior_work/aopw_bloom_response_by_study/....csv",  index=False)